In [ ]:
!pip install anvil-uplink
import anvil.server
import anvil.media
anvil.server.connect("NBHWA5ZZVWPTUPHXVAZEMXFY-6ZULEMIPCJ7CEAH2")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import os
import pickle
import wandb

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (32,32,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
#model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(36))
model.add(Activation('softmax'))


model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   brightness_range=[0.7,1.2],
                                   width_shift_range=[-0.1,0.1],
                                 )

test_datagen = ImageDataGenerator(rescale = 1./255)
#                                  shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    brightness_range=[0.6,1.0]
    

train_generator = train_datagen.flow_from_directory(
    directory = '../input/alpha-numeric/mixed/Training_4',
    target_size = (32,32),
    batch_size =14,
    class_mode = 'categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory = '../input/alpha-numeric/mixed/Validation_4',
    target_size = (32,32),
    batch_size = 14,
    class_mode = 'categorical',
    shuffle=False
)

train_steps_per_epoch = np.math.ceil(train_generator.samples / train_generator.batch_size)
#train_steps_per_epoch = 64
print (train_steps_per_epoch)

test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
#test_steps_per_epoch = 60
print (test_steps_per_epoch)


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch =train_steps_per_epoch,
                    epochs = 30,
                    shuffle = True,
                    validation_data  = test_generator,
                    validation_steps = test_steps_per_epoch)

In [ ]:
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('training and validation accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()    
    
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('training and validation loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()  

In [ ]:
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

In [ ]:
true_classes = test_generator.classes
print(true_classes)

In [ ]:
class_labels = list(test_generator.class_indices.keys())
print(class_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,plot_confusion_matrix
from sklearn.metrics import classification_report
conf_mat = confusion_matrix(true_classes, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat,
                              display_labels=class_labels) 
fig,ax=plt.subplots(figsize=(12,12)) 
disp.plot(ax=ax) 

In [ ]:
report=classification_report(true_classes, predicted_classes, target_names=class_labels,output_dict=True)
df = pd.DataFrame(report).transpose()
df

In [ ]:
from IPython.display import HTML
html = df.to_html()
# write html to file
text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

In [ ]:

with wandb.init(project="img_alpha_numeric_classifier",save_code=True) as run: 
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()    
  wandb.log({"Accuracy-metric": plt}) 
   
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()

  wandb.log({"Loss-metric": plt})

  model.save(os.path.join(wandb.run.dir, "model.h5"))
  
   
  disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat,display_labels=class_labels) 
  disp=disp.plot()
  wandb.log({"conf_mat" : plt})

  wandb.log({"Classification_report": wandb.Html(open("./index.html"))})
wandb.finish()

In [ ]:
model.save("hybrid_cnn_model.h5")

In [ ]:
model = keras.models.load_model("hybrid_cnn_model.h5")

In [ ]:
def get_result(result):
    if result[0][0] == 1:
        return('0')
    elif result[0][1] == 1:
        return ('1')
    elif result[0][2] == 1:
        return ('2')
    elif result[0][3] == 1:
        return ('3')
    elif result[0][4] == 1:
        return ('4')
    elif result[0][5] == 1:
        return ('5')
    elif result[0][6] == 1:
        return ('6')
    elif result[0][7] == 1:
        return ('7')
    elif result[0][8] == 1:
        return ('8')
    elif result[0][9] == 1:
        return ('9')
    elif result[0][10] == 1:
        return('a')
    elif result[0][11] == 1:
        return ('b')
    elif result[0][12] == 1:
        return ('c')
    elif result[0][13] == 1:
        return ('d')
    elif result[0][14] == 1:
        return ('e')
    elif result[0][15] == 1:
        return ('f')
    elif result[0][16] == 1:
        return ('g')
    elif result[0][17] == 1:
        return ('h')
    elif result[0][18] == 1:
        return ('i')
    elif result[0][19] == 1:
        return ('j')
    elif result[0][20] == 1:
        return ('k')
    elif result[0][21] == 1:
        return ('l')
    elif result[0][22] == 1:
        return ('m')
    elif result[0][23] == 1:
        return ('n')
    elif result[0][24] == 1:
        return ('o')
    elif result[0][25] == 1:
        return ('p')
    elif result[0][26] == 1:
        return ('q')
    elif result[0][27] == 1:
        return ('r')
    elif result[0][28] == 1:
        return ('s')
    elif result[0][29] == 1:
        return ('t')
    elif result[0][30] == 1:
        return ('u')
    elif result[0][31] == 1:
        return ('v')
    elif result[0][32] == 1:
        return ('w')
    elif result[0][33] == 1:
        return ('x')
    elif result[0][34] == 1:
        return ('y')
    elif result[0][35] == 1:
        return ('z')

In [ ]:
filename = r'../input/alpha-numeric/mixed/Validation_4/p/images.png'
test_image = image.load_img(filename, target_size = (32,32))
plt.imshow(test_image)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
result = model.predict(test_image)
result = get_result(result)
print ('Predicted Alphabet is: {}'.format(result))

In [ ]:
@anvil.server.callable
def model_run(path):
    with anvil.media.TempFile(path) as filename:
        test_image = image.load_img(filename, target_size = (32,32))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        result = get_result(result)
        return ('Predicted Alphabet is: {}'.format(result))
        